In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
import argparse
import os

import numpy as np
np.random.seed(2016)
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import list_pictures, array_to_img

from image_ext import list_pictures_in_multidir, load_imgs_asarray   

Using TensorFlow backend.


In [2]:
def create_fcn(input_size):
    inputs = Input((3, input_size[1], input_size[0]))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv5)

    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool5)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv6)

    up7 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv6), conv5])
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(up7)
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv7)

    up8 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv7), conv4])
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv8)

    up9 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv8), conv3])
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    up10 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv9), conv2])
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv10)

    up11 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv10), conv1])
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid', data_format='channels_first')(conv11)
    #conv12 = Conv2D(1, 1, 1)(conv11)
    
    fcn = Model(input=inputs, output=conv12)

    return fcn

In [3]:
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return (2.*intersection + 1) / (K.sum(y_true) + K.sum(y_pred) + 1)

In [4]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [5]:
if __name__ == '__main__':

    fpaths_xs_train = list_pictures_in_multidir(['./dataset/train/image/', 
                                                 './dataset/train/image-aug3/'])
    fpaths_ys_train = list_pictures_in_multidir(['./dataset/train/gt/', 
                                                 './dataset/train/gt-aug3/'])
    fpaths_xs_valid = list_pictures_in_multidir(['./dataset/valid/image/', 
                                                 './dataset/valid/image-aug3/'])
    fpaths_ys_valid = list_pictures_in_multidir(['./dataset/valid/gt/', 
                                                 './dataset/valid/gt-aug3/'])
#    fpaths_xs_train = list_pictures_in_multidir(['./data-scene/train/', 
#                                                 './data-scene/train-aug/'])
#    fpaths_ys_train = list_pictures_in_multidir(['./data-scene/train_mask/', 
#                                                 './data-scene/train_mask-aug/'])
#    fpaths_xs_valid = list_pictures_in_multidir(['./data-scene/valid/', 
#                                                 './data-scene/valid-aug'])
#    fpaths_ys_valid = list_pictures_in_multidir(['./data-scene/valid_mask', 
#                                                 './data-scene/valid_mask-aug/'])
    
    #dim_ordering ='tf' 
    dim_ordering = 'channels_first'
    target_size = (224, 224)
    
    # データを配列として取得
    print('loading data...')
    X_train = load_imgs_asarray(fpaths_xs_train, grayscale=False, 
                                target_size=target_size, dim_ordering=dim_ordering)
    Y_train = load_imgs_asarray(fpaths_ys_train, grayscale=True, 
                                target_size=target_size, dim_ordering=dim_ordering)
    X_valid = load_imgs_asarray(fpaths_xs_valid, grayscale=False,
                                target_size=target_size, dim_ordering=dim_ordering)
    Y_valid = load_imgs_asarray(fpaths_ys_valid, grayscale=True,
                                target_size=target_size, dim_ordering=dim_ordering)
    print('==> ' + str(len(X_train)) + ' training images loaded')
    print('==> ' + str(len(Y_train)) + ' training masks loaded')
    print('==> ' + str(len(X_valid)) + ' validation images loaded')
    print('==> ' + str(len(Y_valid)) + ' validation masks loaded')    

loading data...
./dataset/train/image/ueno0267.png
./dataset/train/image/kitazumi0578.png
./dataset/train/image/mitsuzumi0398.png
./dataset/train/image/matsuda0238.png
./dataset/train/image/ueno0379.png
./dataset/train/image/matsuda0499.png
./dataset/train/image/ueno0615.png
./dataset/train/image/kitazumi0231.png
./dataset/train/image/higuchi0386.png
./dataset/train/image/kitazumi0670.png
./dataset/train/image/fujiwara0509.png
./dataset/train/image/fujiwara0892.png
./dataset/train/image/matsuda0431.png
./dataset/train/image/ueno0526.png
./dataset/train/image/mitsuzumi0634.png
./dataset/train/image/higuchi0679.png
./dataset/train/image/matsuda0747.png
./dataset/train/image/mitsuzumi0583.png
./dataset/train/image/kitazumi0443.png
./dataset/train/image/higuchi0727.png
./dataset/train/image/mitsuzumi0756.png
./dataset/train/image/kitazumi0685.png
./dataset/train/image/matsuda0790.png
./dataset/train/image/fujiwara0757.png
./dataset/train/image/kitazumi0054.png
./dataset/train/image/higuchi

./dataset/train/image-aug3/00852.jpg
./dataset/train/image-aug3/01757.jpg
./dataset/train/image-aug3/00082.jpg
./dataset/train/image-aug3/01977.jpg
./dataset/train/image-aug3/00910.jpg
./dataset/train/image-aug3/01501.jpg
./dataset/train/image-aug3/00120.jpg
./dataset/train/image-aug3/00257.jpg
./dataset/train/image-aug3/01420.jpg
./dataset/train/image-aug3/00709.jpg
./dataset/train/image-aug3/01138.jpg
./dataset/train/image-aug3/01419.jpg
./dataset/train/image-aug3/00947.jpg
./dataset/train/image-aug3/01849.jpg
./dataset/train/image-aug3/00495.jpg
./dataset/train/image-aug3/01714.jpg
./dataset/train/image-aug3/01878.jpg
./dataset/train/image-aug3/00096.jpg
./dataset/train/image-aug3/00202.jpg
./dataset/train/image-aug3/00166.jpg
./dataset/train/image-aug3/01437.jpg
./dataset/train/image-aug3/00394.jpg
./dataset/train/image-aug3/00090.jpg
./dataset/train/image-aug3/01012.jpg
./dataset/train/image-aug3/01322.jpg
./dataset/train/image-aug3/00175.jpg
./dataset/train/image-aug3/00168.jpg
.

./dataset/train/image-aug3/01868.jpg
./dataset/train/image-aug3/00772.jpg
./dataset/train/image-aug3/00836.jpg
./dataset/train/image-aug3/00540.jpg
./dataset/train/image-aug3/01494.jpg
./dataset/train/image-aug3/01595.jpg
./dataset/train/image-aug3/00607.jpg
./dataset/train/image-aug3/00423.jpg
./dataset/train/image-aug3/00421.jpg
./dataset/train/image-aug3/00783.jpg
./dataset/train/image-aug3/01157.jpg
./dataset/train/image-aug3/00016.jpg
./dataset/train/image-aug3/00712.jpg
./dataset/train/image-aug3/01402.jpg
./dataset/train/image-aug3/01120.jpg
./dataset/train/image-aug3/01537.jpg
./dataset/train/image-aug3/00548.jpg
./dataset/train/image-aug3/00960.jpg
./dataset/train/image-aug3/01389.jpg
./dataset/train/image-aug3/01705.jpg
./dataset/train/image-aug3/00873.jpg
./dataset/train/image-aug3/00986.jpg
./dataset/train/image-aug3/01263.jpg
./dataset/train/image-aug3/01994.jpg
./dataset/train/image-aug3/01817.jpg
./dataset/train/image-aug3/01618.jpg
./dataset/train/image-aug3/01911.jpg
.

./dataset/train/image-aug3/00968.jpg
./dataset/train/image-aug3/01394.jpg
./dataset/train/image-aug3/01099.jpg
./dataset/train/image-aug3/00891.jpg
./dataset/train/image-aug3/01269.jpg
./dataset/train/image-aug3/01266.jpg
./dataset/train/image-aug3/01009.jpg
./dataset/train/image-aug3/01874.jpg
./dataset/train/image-aug3/00249.jpg
./dataset/train/image-aug3/01902.jpg
./dataset/train/image-aug3/01596.jpg
./dataset/train/image-aug3/01386.jpg
./dataset/train/image-aug3/00193.jpg
./dataset/train/image-aug3/01824.jpg
./dataset/train/image-aug3/01609.jpg
./dataset/train/image-aug3/00396.jpg
./dataset/train/image-aug3/01803.jpg
./dataset/train/image-aug3/01519.jpg
./dataset/train/image-aug3/01806.jpg
./dataset/train/image-aug3/00325.jpg
./dataset/train/image-aug3/00838.jpg
./dataset/train/image-aug3/01234.jpg
./dataset/train/image-aug3/01941.jpg
./dataset/train/image-aug3/00414.jpg
./dataset/train/image-aug3/00615.jpg
./dataset/train/image-aug3/01296.jpg
./dataset/train/image-aug3/00826.jpg
.

./dataset/train/image-aug3/00347.jpg
./dataset/train/image-aug3/01034.jpg
./dataset/train/image-aug3/01262.jpg
./dataset/train/image-aug3/00717.jpg
./dataset/train/image-aug3/01847.jpg
./dataset/train/image-aug3/00990.jpg
./dataset/train/image-aug3/01598.jpg
./dataset/train/image-aug3/01967.jpg
./dataset/train/image-aug3/01183.jpg
./dataset/train/image-aug3/01608.jpg
./dataset/train/image-aug3/01455.jpg
./dataset/train/image-aug3/01773.jpg
./dataset/train/image-aug3/00889.jpg
./dataset/train/image-aug3/00959.jpg
./dataset/train/image-aug3/01257.jpg
./dataset/train/image-aug3/00658.jpg
./dataset/train/image-aug3/00361.jpg
./dataset/train/image-aug3/01405.jpg
./dataset/train/image-aug3/01623.jpg
./dataset/train/image-aug3/01713.jpg
./dataset/train/image-aug3/01404.jpg
./dataset/train/image-aug3/00867.jpg
./dataset/train/image-aug3/00063.jpg
./dataset/train/image-aug3/00213.jpg
./dataset/train/image-aug3/01017.jpg
./dataset/train/image-aug3/01430.jpg
./dataset/train/image-aug3/01800.jpg
.

...

./dataset/train/gt-aug3/00105.jpg
./dataset/train/gt-aug3/01347.jpg
./dataset/train/gt-aug3/00118.jpg
./dataset/train/gt-aug3/00835.jpg
./dataset/train/gt-aug3/01340.jpg
./dataset/train/gt-aug3/00878.jpg
./dataset/train/gt-aug3/01298.jpg
./dataset/train/gt-aug3/00065.jpg
./dataset/train/gt-aug3/01014.jpg
./dataset/train/gt-aug3/01121.jpg
./dataset/train/gt-aug3/00942.jpg
./dataset/train/gt-aug3/00911.jpg
./dataset/train/gt-aug3/01291.jpg
./dataset/train/gt-aug3/00306.jpg
./dataset/train/gt-aug3/00460.jpg
./dataset/train/gt-aug3/01105.jpg
./dataset/train/gt-aug3/00330.jpg
./dataset/train/gt-aug3/01079.jpg
./dataset/train/gt-aug3/00618.jpg
./dataset/train/gt-aug3/01216.jpg
./dataset/train/gt-aug3/01518.jpg
./dataset/train/gt-aug3/01612.jpg
./dataset/train/gt-aug3/01384.jpg
./dataset/train/gt-aug3/00810.jpg
./dataset/train/gt-aug3/00389.jpg
./dataset/train/gt-aug3/01191.jpg
./dataset/train/gt-aug3/00612.jpg
./dataset/train/gt-aug3/01571.jpg
./dataset/train/gt-aug3/00212.jpg
./dataset/trai

./dataset/train/gt-aug3/00377.jpg
./dataset/train/gt-aug3/01621.jpg
./dataset/train/gt-aug3/00154.jpg
./dataset/train/gt-aug3/00034.jpg
./dataset/train/gt-aug3/01899.jpg
./dataset/train/gt-aug3/01213.jpg
./dataset/train/gt-aug3/00192.jpg
./dataset/train/gt-aug3/00479.jpg
./dataset/train/gt-aug3/00906.jpg
./dataset/train/gt-aug3/00683.jpg
./dataset/train/gt-aug3/00922.jpg
./dataset/train/gt-aug3/01832.jpg
./dataset/train/gt-aug3/00661.jpg
./dataset/train/gt-aug3/00858.jpg
./dataset/train/gt-aug3/00404.jpg
./dataset/train/gt-aug3/00009.jpg
./dataset/train/gt-aug3/01573.jpg
./dataset/train/gt-aug3/00516.jpg
./dataset/train/gt-aug3/00695.jpg
./dataset/train/gt-aug3/00583.jpg
./dataset/train/gt-aug3/01897.jpg
./dataset/train/gt-aug3/01813.jpg
./dataset/train/gt-aug3/00593.jpg
./dataset/train/gt-aug3/00689.jpg
./dataset/train/gt-aug3/00097.jpg
./dataset/train/gt-aug3/00965.jpg
./dataset/train/gt-aug3/00782.jpg
./dataset/train/gt-aug3/01117.jpg
./dataset/train/gt-aug3/01032.jpg
./dataset/trai

./dataset/train/gt-aug3/00294.jpg
./dataset/train/gt-aug3/01932.jpg
./dataset/train/gt-aug3/00559.jpg
./dataset/train/gt-aug3/00693.jpg
./dataset/train/gt-aug3/01095.jpg
./dataset/train/gt-aug3/01617.jpg
./dataset/train/gt-aug3/01552.jpg
./dataset/train/gt-aug3/00316.jpg
./dataset/train/gt-aug3/01927.jpg
./dataset/train/gt-aug3/01440.jpg
./dataset/train/gt-aug3/00224.jpg
./dataset/train/gt-aug3/00900.jpg
./dataset/train/gt-aug3/00092.jpg
./dataset/train/gt-aug3/00410.jpg
./dataset/train/gt-aug3/01016.jpg
./dataset/train/gt-aug3/00238.jpg
./dataset/train/gt-aug3/01947.jpg
./dataset/train/gt-aug3/00484.jpg
./dataset/train/gt-aug3/00796.jpg
./dataset/train/gt-aug3/00112.jpg
./dataset/train/gt-aug3/00356.jpg
./dataset/train/gt-aug3/00703.jpg
./dataset/train/gt-aug3/01256.jpg
./dataset/train/gt-aug3/00248.jpg
./dataset/train/gt-aug3/01859.jpg
./dataset/train/gt-aug3/00437.jpg
./dataset/train/gt-aug3/01168.jpg
./dataset/train/gt-aug3/00139.jpg
./dataset/train/gt-aug3/00775.jpg
./dataset/trai

./dataset/train/gt-aug3/00301.jpg
./dataset/train/gt-aug3/01311.jpg
./dataset/train/gt-aug3/01822.jpg
./dataset/train/gt-aug3/00985.jpg
./dataset/train/gt-aug3/00040.jpg
./dataset/train/gt-aug3/01954.jpg
./dataset/train/gt-aug3/01153.jpg
./dataset/train/gt-aug3/01869.jpg
./dataset/train/gt-aug3/01199.jpg
./dataset/train/gt-aug3/00987.jpg
./dataset/train/gt-aug3/00358.jpg
./dataset/train/gt-aug3/01029.jpg
./dataset/train/gt-aug3/00146.jpg
./dataset/train/gt-aug3/01979.jpg
./dataset/train/gt-aug3/00638.jpg
./dataset/train/gt-aug3/01372.jpg
./dataset/train/gt-aug3/01607.jpg
./dataset/train/gt-aug3/01945.jpg
./dataset/train/gt-aug3/00012.jpg
./dataset/train/gt-aug3/01107.jpg
./dataset/train/gt-aug3/01587.jpg
./dataset/train/gt-aug3/01951.jpg
./dataset/train/gt-aug3/00156.jpg
./dataset/train/gt-aug3/01548.jpg
./dataset/train/gt-aug3/00351.jpg
./dataset/train/gt-aug3/01777.jpg
./dataset/train/gt-aug3/01599.jpg
./dataset/train/gt-aug3/00266.jpg
./dataset/train/gt-aug3/00585.jpg
./dataset/trai

./dataset/valid/image-aug3/00003.jpg
./dataset/valid/image-aug3/00199.jpg
./dataset/valid/image-aug3/00108.jpg
./dataset/valid/image-aug3/00174.jpg
./dataset/valid/image-aug3/00075.jpg
./dataset/valid/image-aug3/00017.jpg
./dataset/valid/image-aug3/00140.jpg
./dataset/valid/image-aug3/00081.jpg
./dataset/valid/image-aug3/00032.jpg
./dataset/valid/image-aug3/00107.jpg
./dataset/valid/image-aug3/00054.jpg
./dataset/valid/image-aug3/00172.jpg
./dataset/valid/image-aug3/00088.jpg
./dataset/valid/image-aug3/00068.jpg
./dataset/valid/image-aug3/00111.jpg
./dataset/valid/image-aug3/00124.jpg
./dataset/valid/image-aug3/00072.jpg
./dataset/valid/image-aug3/00070.jpg
./dataset/valid/image-aug3/00074.jpg
./dataset/valid/image-aug3/00051.jpg
./dataset/valid/image-aug3/00169.jpg
./dataset/valid/image-aug3/00116.jpg
./dataset/valid/image-aug3/00027.jpg
./dataset/valid/image-aug3/00040.jpg
./dataset/valid/image-aug3/00146.jpg
./dataset/valid/image-aug3/00012.jpg
./dataset/valid/image-aug3/00156.jpg
.

In [6]:
    # 前処理
    print('computing mean and standard deviation...')
    mean = np.mean(X_train, axis=(0, 2, 3))
    std = np.std(X_train, axis=(0, 2, 3))
    print('==> mean: ' + str(mean))
    print('==> std : ' + str(std))
    
    fname_stats = 'stats.npy'

    print('saving mean and standard deviation to ' + fname_stats + '...')
    stats = {'mean': mean, 'std': std}
    np.savez(fname_stats, **stats)
    print('==> done')

    print('globally normalizing data...')
    for i in range(3):
        X_train[:, i] = (X_train[:, i] - mean[i]) / std[i]
        X_valid[:, i] = (X_valid[:, i] - mean[i]) / std[i]
    Y_train /= 255
    Y_valid /= 255
    print('==> done')
    
    print(mean.shape)
    print(std.shape)

computing mean and standard deviation...
==> mean: [ 205.33253479  170.61235046  186.54370117]
==> std : [ 60.46948624  61.61787415  61.97478104]
saving mean and standard deviation to stats.npy...
==> done
globally normalizing data...
==> done
(3,)
(3,)


In [7]:
    # モデルを作成
    print('creating model...')
    model = create_fcn(target_size)
    #model.summary()

creating model...


/home/nakazawa_atsushi/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=Tensor("in...)`


In [8]:
# 損失関数，最適化手法を定義
adam = Adam(lr=1e-5)
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])

# 構造・重みを保存するディレクトリーの有無を確認
dpath_checkpoints = './checkpoints/'
if not os.path.isdir(dpath_checkpoints):
    os.mkdir(dpath_checkpoints)

In [9]:
# モデルの構造を保存
json_string = model.to_json()
fname_architecture = 'fcn.json'
fpath_architecture = os.path.join(dpath_checkpoints, fname_architecture)
with open(fpath_architecture, 'w', encoding='utf8') as f:
    f.write(json_string)

In [10]:
print(X_train.shape)
print(Y_train.shape)

(2074, 3, 224, 224)
(2074, 1, 224, 224)


In [12]:
# 重みを保存するためのオブジェクトを用意
fname_weights = 'model_weights_{epoch:02d}.h5'
fpath_weights = os.path.join(dpath_checkpoints, fname_weights)
checkpointer = ModelCheckpoint(filepath=fpath_weights, save_best_only=False)

# トレーニングを開始
print('start training...')
history = model.fit(X_train, Y_train, batch_size=5, epochs=140, verbose=1, 
          shuffle=True, validation_data=(X_valid, Y_valid), callbacks=[checkpointer])

start training...
Train on 2074 samples, validate on 212 samples
Epoch 1/140
2074/2074 [==============================] - 183s - loss: -0.2174 - dice_coef: 0.2174 - val_loss: -0.3863 - val_dice_coef: 0.3863
Epoch 2/140
2074/2074 [==============================] - 180s - loss: -0.3367 - dice_coef: 0.3367 - val_loss: -0.4051 - val_dice_coef: 0.4051
Epoch 3/140
2074/2074 [==============================] - 180s - loss: -0.3521 - dice_coef: 0.3521 - val_loss: -0.3921 - val_dice_coef: 0.3921
Epoch 4/140
2074/2074 [==============================] - 180s - loss: -0.3652 - dice_coef: 0.3652 - val_loss: -0.4161 - val_dice_coef: 0.4161
Epoch 5/140
2074/2074 [==============================] - 180s - loss: -0.3682 - dice_coef: 0.3682 - val_loss: -0.3992 - val_dice_coef: 0.3992
Epoch 6/140
2074/2074 [==============================] - 180s - loss: -0.3761 - dice_coef: 0.3761 - val_loss: -0.4209 - val_dice_coef: 0.4209
Epoch 7/140
2074/2074 [==============================] - 180s - loss: -0.3793 - dic

2074/2074 [==============================] - 180s - loss: -0.4356 - dice_coef: 0.4356 - val_loss: -0.4273 - val_dice_coef: 0.4273
Epoch 58/140
2074/2074 [==============================] - 180s - loss: -0.4389 - dice_coef: 0.4389 - val_loss: -0.4326 - val_dice_coef: 0.4326
Epoch 59/140
2074/2074 [==============================] - 180s - loss: -0.4345 - dice_coef: 0.4345 - val_loss: -0.4347 - val_dice_coef: 0.4347
Epoch 60/140
2074/2074 [==============================] - 180s - loss: -0.4392 - dice_coef: 0.4392 - val_loss: -0.4279 - val_dice_coef: 0.4279
Epoch 61/140
2074/2074 [==============================] - 180s - loss: -0.4398 - dice_coef: 0.4398 - val_loss: -0.4266 - val_dice_coef: 0.4266
Epoch 62/140
2074/2074 [==============================] - 180s - loss: -0.4372 - dice_coef: 0.4372 - val_loss: -0.4257 - val_dice_coef: 0.4257
Epoch 63/140
2074/2074 [==============================] - 180s - loss: -0.4400 - dice_coef: 0.4400 - val_loss: -0.4294 - val_dice_coef: 0.4294
Epoch 64/140

KeyboardInterrupt: 